# Import Packages

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
%matplotlib inline

# Load Data

In [2]:
df = pd.read_csv("D:\\Python\Quant Equity Investing 9733\Project\Test\Merged_Cleansed\df_train.csv")
SPY = pd.read_csv("D:\\Python\Quant Equity Investing 9733\Project\Test\Merged_Cleansed\SPY_train.csv")
Rf = pd.read_csv("D:\\Python\Quant Equity Investing 9733\Project\Test\Merged_Cleansed\Risk_Free.csv")

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
df.head(3)

,PERMNO,Date,TICKER,Price,Return,Shares Outstanding,Turnover,Dollar Turnover,Current Ratio,Cash Ratio,Quick Ratio,Revenue Growth,COGS Growth,Operating Income Growth,Net Income Growth,EPS Basic Growth,Net Operating Cash Flow Growth
0,10051,1999-01-01,HGR,22.5000,0.111111,18179,0.701029,15.773145,3.014681,0.392692,2.327885,0.040000,0.030781,0.085278,0.138436,0.043478,0.593825
1,10051,1999-02-01,HGR,25.0000,-0.392500,18179,0.526817,13.170416,3.014681,0.392692,2.327885,0.040000,0.030781,0.085278,0.138436,0.043478,0.593825
2,10051,1999-03-01,HGR,15.1875,-0.111111,18179,1.615710,24.538602,3.161060,0.215989,2.402887,0.054699,0.016057,0.088484,0.069417,0.000000,0.329531


In [4]:
SPY.head(3)

,Date,Return
0,1995-01-01,0.012817
1,1995-02-01,0.024591
2,1995-03-01,0.032419


In [5]:
Rf.head(3)

,Date,Rf
0,1990-01-01,0.006958
1,1990-02-01,0.007033
2,1990-03-01,0.007208


# Rolling Window and Calculate Var & Cov

In [6]:
SPY_ = SPY.copy().rename(columns = {'Return': 'SPY Return'})
df = pd.merge(df, SPY_, on = 'Date', how = 'outer')
df = pd.merge(df, Rf, on = 'Date', how = 'outer')

In [7]:
# # Transform Date into datetime type
# df['Date'] = pd.to_datetime(df['Date'])
# df.set_index('Date', inplace=True)

In [8]:
df.dropna(subset=['TICKER'], inplace=True)

In [9]:
df.head(3)

,PERMNO,Date,TICKER,Price,Return,Shares Outstanding,Turnover,Dollar Turnover,Current Ratio,Cash Ratio,Quick Ratio,Revenue Growth,COGS Growth,Operating Income Growth,Net Income Growth,EPS Basic Growth,Net Operating Cash Flow Growth,SPY Return,Rf
0,10051.0,1999-01-01,HGR,22.5000,0.111111,18179.0,0.701029,15.773145,3.014681,0.392692,2.327885,0.040000,0.030781,0.085278,0.138436,0.043478,0.593825,0.044942,0.003792
1,10123.0,1999-01-01,AK,18.2500,-0.013699,20576.0,0.078101,1.425338,1.263811,0.077769,1.263811,-0.365916,-0.403071,-0.194629,-0.989821,-0.985915,-7.794844,0.044942,0.003792
2,10145.0,1999-01-01,ALD,44.3125,-0.119887,560050.0,0.508451,22.530736,1.078689,0.137319,0.628930,-0.033083,-0.040357,-0.007541,-0.060000,-0.048387,0.642570,0.044942,0.003792


In [32]:
def rolling(df_):
    df = df_.copy()
    #df.sort_values(by = ['PERMNO'])
    TICKER = list(set(df['TICKER']))
    #df.set_index('TICKER', inplace=True)
    #dataframe = df.loc[TICKER[0]]
    dataframe = df[df['TICKER'] == TICKER[0]].copy()
    dataframe['Var'] = dataframe['SPY Return'].rolling(window=24).var()
    dataframe['Cov'] = dataframe['Return'].rolling(window=24).cov(dataframe['SPY Return'])
    dataframe['Beta'] = dataframe['Cov'] / dataframe['Var']
    dataframe['Alpha'] = dataframe['Return'] - dataframe['Rf'] - dataframe['Beta'] * (dataframe['SPY Return'] - dataframe['Rf'])
    
    for i in range(1, len(TICKER)):
        #DF = df.loc[TICKER[i]]
        DF = df[df['TICKER'] == TICKER[i]].copy()
        if len(DF) < 24:
            continue
        DF['Var'] = DF['SPY Return'].rolling(window=24).var()
        DF['Cov'] = DF['Return'].rolling(window=24).cov(DF['SPY Return'])
        DF['Beta'] = DF['Cov'] / DF['Var']
        DF['Alpha'] = DF['Return'] - DF['Rf'] - DF['Beta'] * (DF['SPY Return'] - DF['Rf']) 
        
        dataframe = pd.concat([dataframe, DF])
        
    return dataframe

In [33]:
df_ = rolling(df)

In [34]:
df_ 

,PERMNO,Date,TICKER,Price,Return,Shares Outstanding,Turnover,Dollar Turnover,Current Ratio,Cash Ratio,Quick Ratio,Revenue Growth,COGS Growth,Operating Income Growth,Net Income Growth,EPS Basic Growth,Net Operating Cash Flow Growth,SPY Return,Rf,Var,Cov,Beta,Alpha
660,75844.0,1999-01-01,POT,63.8750,-0.049130,54241.0,0.563817,36.013813,1.739693,0.152740,0.920842,-0.210577,-0.204199,-0.256408,-0.386155,-0.384146,0.000000,0.044942,0.003792,NaN,NaN,NaN,NaN
1850,75844.0,1999-02-01,POT,60.5000,-0.063017,54253.0,0.616427,37.293818,1.739693,0.152740,0.920842,-0.210577,-0.204199,-0.256408,-0.386155,-0.384146,0.000000,0.036561,0.004338,NaN,NaN,NaN,NaN
3033,75844.0,1999-03-01,POT,56.6875,-0.056229,54253.0,0.782519,44.359042,1.662684,0.071349,0.854213,0.039843,0.062942,-0.007179,-0.009707,-0.009901,0.000000,-0.028939,0.004244,NaN,NaN,NaN,NaN
4216,75844.0,1999-04-01,POT,53.5000,0.135270,54253.0,0.669640,35.825761,1.662684,0.071349,0.854213,0.039843,0.062942,-0.007179,-0.009707,-0.009901,0.000000,0.046563,0.004348,NaN,NaN,NaN,NaN
5389,75844.0,1999-05-01,POT,60.5000,-0.101240,54253.0,0.532929,32.242208,1.662684,0.071349,0.854213,0.039843,0.062942,-0.007179,-0.009707,-0.009901,0.000000,0.047081,0.004642,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301479,58975.0,1997-08-01,OII,21.6875,0.074928,23094.0,1.515415,32.865568,1.421066,0.247058,1.421066,0.034425,0.157033,-0.419669,-0.424629,-0.428571,-0.389955,0.062972,0.005177,0.002429,-0.001166,-0.479895,0.097486
302583,58975.0,1997-09-01,OII,23.3125,0.021448,23281.0,1.031657,24.050497,1.571464,0.175757,1.571464,-0.022536,-0.060457,0.288410,0.537906,0.625000,-0.025770,-0.020652,0.004993,0.002504,-0.001193,-0.476412,0.004237
303706,58975.0,1997-10-01,OII,23.8125,0.041995,23281.0,0.837636,19.946203,1.571464,0.175757,1.571464,-0.022536,-0.060457,0.288410,0.537906,0.625000,-0.025770,0.030003,0.004755,0.002469,-0.001239,-0.501832,0.049910
304831,58975.0,1997-11-01,OII,24.8125,-0.168766,23281.0,1.076844,26.719186,1.571464,0.175757,1.571464,-0.022536,-0.060457,0.288410,0.537906,0.625000,-0.025770,-0.017186,0.004847,0.002536,-0.000930,-0.366556,-0.181689


In [37]:
df_.to_csv("df_train.csv", index=False)